# How to load data
This notebook is to show how to load data from files

## Import packages

In [1]:
from pathlib import Path
from utils import check_integrity, datestring_to_timestamp, load_data, get_sorted_fluctuation_coins

## Set parameters and read data

In [2]:
data_path = "../data"
start_from = "2021-01-11"
end_before = "2021-03-15"
interval = "3h"

In [3]:
start_from_timestamp = datestring_to_timestamp(start_from)
end_before_timestamp = datestring_to_timestamp(end_before)
data_interval_path_list_raw = list(Path(data_path, interval).iterdir())
data_1D_path_list = list(Path(data_path, '1D').iterdir())

coins_1D_pass = []
for file_path in data_1D_path_list:
    _coin_code = file_path.name.split("_")[0]
    if check_integrity(start_from, end_before, file_path) is not None:
        coins_1D_pass.append(_coin_code)
print(f"{len(coins_1D_pass)} coins have all 1D data")

73 coins have all 1D data


In [4]:
data_interval_path_list = [_p for _p in data_interval_path_list_raw if _p.name.split('_')[0] in coins_1D_pass]

**Load the data of one coin from the given period**

In [5]:
_file_path = data_interval_path_list[0]

_data_df =  load_data(start_from_timestamp, end_before_timestamp, _file_path)
_data_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 505 entries, (1610323200000, 'BTC') to (1615766400000, 'BTC')
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   close     505 non-null    float64            
 1   volume    505 non-null    float64            
 2   datetime  505 non-null    datetime64[ns, UTC]
 3   is_fill   505 non-null    bool               
dtypes: bool(1), datetime64[ns, UTC](1), float64(2)
memory usage: 101.4+ KB


**Load another data of one coin from the given period then concatenate the data**

In [6]:
_file_path_2 = data_interval_path_list[9]

_data_df_2 =  load_data(start_from_timestamp, end_before_timestamp, _file_path_2)
_data_df_2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 503 entries, (1610323200000, 'MKR') to (1615766400000, 'MKR')
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   close     503 non-null    float64            
 1   volume    503 non-null    float64            
 2   datetime  503 non-null    datetime64[ns, UTC]
 3   is_fill   503 non-null    bool               
dtypes: bool(1), datetime64[ns, UTC](1), float64(2)
memory usage: 100.6+ KB


**Load and fill null values**

In [7]:
_data_df_2_filled =  load_data(start_from_timestamp, end_before_timestamp, _file_path_2, fill_na=True)
_data_df_2_filled.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 505 entries, (1610323200000, 'MKR') to (1615766400000, 'MKR')
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   close     505 non-null    float64            
 1   volume    505 non-null    float64            
 2   datetime  505 non-null    datetime64[ns, UTC]
 3   is_fill   505 non-null    bool               
dtypes: bool(1), datetime64[ns, UTC](1), float64(2)
memory usage: 50.3+ KB


**Concatenate dataframes**

In [8]:
_new_data_df = _data_df.append(_data_df_2)
_new_data_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1008 entries, (1610323200000, 'BTC') to (1615766400000, 'MKR')
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   close     1008 non-null   float64            
 1   volume    1008 non-null   float64            
 2   datetime  1008 non-null   datetime64[ns, UTC]
 3   is_fill   1008 non-null   bool               
dtypes: bool(1), datetime64[ns, UTC](1), float64(2)
memory usage: 47.9+ KB


**Sorted coins by fluctuation**

In [9]:
"""
get_sorted_fluctuation_coins(
    start_from (str, int): the timestamp or datestring of the first day
    end_before (str, int): the timestamp or datestring of the last day
    data_dir   (str):   the path if directory which has .csv files
    normalize_price (bool): use MinMaxScaler() to normalize the close price if True
    incl_coins (list, None): only return the coins in this provided list; None to return all coins
    return_details (bool): return the standard deviations in a DataFrame if True; False to only return a list with coin codes
    [DEPRECATED] start_from_timestamp (int): the timestamp of the first day
    [DEPRECATED] end_before_timestamp (int): the timestamp of the last day
)
"""

fluc_df = get_sorted_fluctuation_coins(
    start_from="2021-04-14",
    end_before="2021-06-15",
    data_dir='../data/1D',
    # incl_coins=ONLY_WANT_THESE_COINS,
    normalize_price=False,
    return_details=True,
    interval='1D'
)
fluc_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 119 entries, YFI to PAX
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   close_std   119 non-null    float64
 1   volume_sum  119 non-null    float64
dtypes: float64(2)
memory usage: 2.8+ KB


In [10]:
fluc_df.head()

,close_std,volume_sum
coin,,
YFI,10151.399072,4253.657610
BTC,9443.343583,735776.442452
WBT,9383.947860,11.386429
RBT,9313.638109,13.894120
MKR,804.739281,5306.265545


In [11]:
fluc_df.tail()

,close_std,volume_sum
coin,,
BTT,0.001735,3.321663e+09
KAN,0.001509,3.151322e+07
PAS,0.001380,1.281178e+07
DAI,0.001095,3.376238e+07
PAX,0.000927,3.426721e+07


In [14]:
fluc_df[fluc_df['close_std']>1.]

,close_std,volume_sum
coin,,
YFI,10151.399072,4.253658e+03
BTC,9443.343583,7.357764e+05
WBT,9383.947860,1.138643e+01
RBT,9313.638109,1.389412e+01
MKR,804.739281,5.306266e+03
ETH2X,605.917976,6.689698e+01
ETH,547.771285,7.777233e+06
BCHN,266.827141,5.641582e+05
COMP,166.511896,1.205746e+05


In [12]:
fluc_list = get_sorted_fluctuation_coins(
    start_from="2021-04-14",
    end_before="2021-06-15",
    data_dir='../data/1D',
    # incl_coins=ONLY_WANT_THESE_COINS,
    normalize_price=False,
    return_details=False,
    interval='1D'
)
fluc_list[:5]

['YFI', 'BTC', 'WBT', 'RBT', 'MKR']

In [13]:
fluc_list = get_sorted_fluctuation_coins(
    start_from="2021-04-14",
    end_before="2021-06-15",
    data_dir='../data/1D',
    incl_coins="BTC,ETH,AVAX,SOL,LUNA,LTC,ZEC,TRX,DOT,XRP,CTK".split(','),
    normalize_price=False,
    return_details=False,
    interval='1D'
)
fluc_list[:5]

['BTC', 'ETH', 'LTC', 'ZEC', 'AVAX']